In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import math

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cuda = True if torch.cuda.is_available() else False
    
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor    

torch.manual_seed(125)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(125)

In [2]:
import torchvision.transforms as transforms

mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (1.0,))
])

In [3]:
from torchvision.datasets import MNIST

download_root = '../chap07/MNIST_DATASET'
train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

In [4]:
batch_size=64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [5]:
batch_size = 100
n_iters = 6000
num_epochs = n_iters / (len(train_dataset)/batch_size)
num_epochs = int(num_epochs)

In [6]:
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, bias=True):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias

        self.x2h = nn.Linear(input_size, 4 * hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 4*hidden_size, bias=bias)
        ### 은닉층의 뉴런에 4를 곱하는 이유?
        ##### 계산된 게이트는 4개의 게이트(망각, 기억, 셀, 출력)으로 쪼개지기 때문
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0/math.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std,std)

    def forward(self, x, hidden):
        hx, cx = hidden
        x = x.view(-1, x.size(1))

        gates = self.x2h(x) + self.h2h(hx)
        gates = gates.squeeze()
        ingate, forgetgate, cellgate, outgate = gates.chunk(4,1) ##4개의 게이트로 쪼갬

        ingate = F.sigmoid(ingate)
        forgetgate = F.sigmoid(forgetgate)
        cellgate = F.tanh(cellgate)
        outgate = F.sigmoid(outgate)

        cy = torch.mul(cx, forgetgate) + torch.mul(ingate,cellgate)
        hy = torch.mul(outgate, F.tanh(cy))
        return (hy,cy)

In [7]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        self.lstm = LSTMCell(input_dim, hidden_dim, layer_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
            c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            c0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))

        outs = []
        cn = c0[0,:,:]
        hn = h0[0,:,:]

        for seq in range(x.size(1)):
            hn, cn = self.lstm(x[:,seq,:], (hn,cn))
            outs.append(hn)

        out = outs[-1].squeeze()
        out = self.fc(out)
        return out


In [8]:
input_dim = 28
hidden_dim=128
layer_dim=1
output_dim=10

model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
model.to(device)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [9]:
seq_dim=28
loss_list = []
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, seq_dim, input_dim).to(device))
        labels = Variable(labels.to(device))
    
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.to(device)

        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
        iter += 1

        if iter%500 == 0:
            correct = 0
            total = 0
            for images, labels in valid_loader:
                images = Variable(images.view(-1, seq_dim, input_dim).to(device))
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                total += labels.size(0)
                correct += (predicted.cpu() == labels.cpu()).sum()
            accuracy = 100 * correct / total
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 2.237457752227783. Accuracy: 21.420000076293945
Iteration: 1000. Loss: 0.9093604683876038. Accuracy: 75.72000122070312
Iteration: 1500. Loss: 0.4703645408153534. Accuracy: 87.77999877929688
Iteration: 2000. Loss: 0.2919614613056183. Accuracy: 93.33000183105469
Iteration: 2500. Loss: 0.14118362963199615. Accuracy: 93.83999633789062
Iteration: 3000. Loss: 0.08196574449539185. Accuracy: 96.5
Iteration: 3500. Loss: 0.09124790877103806. Accuracy: 95.87999725341797
Iteration: 4000. Loss: 0.0626092255115509. Accuracy: 97.12000274658203
Iteration: 4500. Loss: 0.046092260628938675. Accuracy: 97.25
Iteration: 5000. Loss: 0.09187523275613785. Accuracy: 96.94999694824219
Iteration: 5500. Loss: 0.09047114849090576. Accuracy: 97.30000305175781
Iteration: 6000. Loss: 0.02395769953727722. Accuracy: 97.80999755859375
Iteration: 6500. Loss: 0.01368747465312481. Accuracy: 97.55000305175781
Iteration: 7000. Loss: 0.02047237753868103. Accuracy: 97.8499984741211
Iteration: 7500. Loss: 

In [17]:
def evaluate(model, val_iter):
    model.eval()
    corrects, total, total_loss = 0,0,0
    with torch.no_grad():
        for images, labels in val_iter:
            images = images.to(device)  
            labels = labels.to(device)
            images = images.view(-1, seq_dim, input_dim) 

            logits = model(images)  
            loss = F.cross_entropy(logits, labels, reduction="sum")
            preds = logits.argmax(dim=1)
            
            total += labels.size(0)
            corrects += (preds == labels).sum().item()
            total_loss += loss.item()

    avg_loss = total_loss / len(val_iter.dataset)
    avg_accuracy = corrects / total
    return avg_loss, avg_accuracy

In [18]:
test_loss, test_accuracy = evaluate(model, test_loader)
print('Test Loss: {}. Test Accuracy: {}'.format(test_loss, test_accuracy))

Test Loss: 0.05972701856642962. Test Accuracy: 0.9804
